# 2.7 Optimización Básica de Consultas

**Resumen Ejecutivo**: Aprende patrones simples de optimización: proyectar solo columnas necesarias, filtrar eficientemente, diseñar índices lógicos y evitar sobrecarga de JOINs.

---

## 🎯 Objetivos de Aprendizaje
- Aplicar mejores prácticas: evitar `SELECT *`, filtros por rango, JOINs solo cuando necesarios.
- Diseñar índices compuestos para consultas frecuentes.
- Identificar anti-patrones (funciones en columnas indexadas, sobre-indexación).

## 🧰 Prerrequisitos
- Haber completado `06_segmentacion_basica.ipynb`.
- Conocer planes de ejecución conceptualmente.
- Familiaridad con índices clustered/non-clustered.

---

## 📚 Conceptos Clave
- **Proyección selectiva**: traer solo columnas usadas reduce I/O y memoria.
- **Filtros eficientes**: rangos (`BETWEEN`) aprovechan índices; funciones en columnas los rompen.
- **Índices compuestos**: orden de columnas importa (filtro → JOIN → ORDER BY).
- **JOINs innecesarios**: si no necesitas datos de la tabla, no la unas.

> ⚠️ Sobre-indexar degrada INSERT/UPDATE; funciones en `WHERE` sobre columnas indexadas anulan el índice.

In [ ]:
-- Evitar SELECT *: seleccionar columnas necesarias
SELECT venta_id, fecha, cliente_id, producto_id, cantidad
FROM dbo.fact_ventas;

In [ ]:
-- Filtro eficiente: venta por rango de fechas (usa índice si existe por fecha)
SELECT venta_id, fecha, cantidad
FROM dbo.fact_ventas
WHERE fecha BETWEEN '2024-01-01' AND '2024-12-31';

🟢 Ejercicio: Reescribir una consulta que hace JOIN innecesario para solo contar ventas totales.

In [ ]:
-- Antes (innecesario): SELECT COUNT(*) FROM fact_ventas fv JOIN dim_productos p ON fv.producto_id = p.producto_id;
-- Después:
SELECT COUNT(*) AS ventas_totales FROM dbo.fact_ventas;

🟠 Ejercicio: Evaluar impacto de filtrar por producto_id vs sin filtro (estimación conceptual).

🔴 Reto: Diseñar índice compuesto lógico para fact_ventas (fecha, producto_id) y escribir consulta que lo aproveche con filtro de ambos y agregación.

In [ ]:
-- Sugerencia índice (no crear sin análisis):
-- CREATE INDEX IX_fact_ventas_fecha_producto ON dbo.fact_ventas(fecha, producto_id);
-- Consulta que lo aprovecharía:
SELECT producto_id, SUM(cantidad) AS unidades
FROM dbo.fact_ventas
WHERE fecha BETWEEN '2024-01-01' AND '2024-12-31' AND producto_id IN (101,102)
GROUP BY producto_id;

Errores comunes: sobre-indexar; usar funciones sobre columnas indexadas en filtros; traer columnas no usadas (penaliza IO).

# Créditos

Este material fue revisado y enriquecido parcialmente mediante asistencia de IA (OpenAI y Claude); la validación y decisiones editoriales finales son humanas.

---
## Navegación
[⬅️ Anterior](06_segmentacion_basica.ipynb) | [Siguiente ➡️](08_control_version_datos.ipynb)
---
